In [1]:
import logging
import requests
from pprint import pprint
from time import time
from gector.gec_model import GecBERTModel

import spacy
from utils.helpers import *

nlp = spacy.load("en")

In [2]:
model = GecBERTModel(
    vocab_path = "./data/output_vocabulary",
    model_paths = ["./pretrain/roberta_1_gector.th"],
    model_name = "roberta",
    is_ensemble = False,
    min_probability = 0.1,
    min_error_probability = 0.1,
)

In [3]:
model.vocab.get_token_from_index(5000, namespace='labels')

'@@UNKNOWN@@'

In [40]:
text = "England coach Andy Robinson is already without centre Will Greenwood and flanker Richard Hill while fly-half Jonny Wilkinson is certain to miss at least the first two games."
batch = []

doc = nlp(text)
sentences = []
for sent in doc.sents:
    tokens = []
    sentences.append(sent.text)
    for token in nlp(sent.text):
        tokens.append(token.text)
    tokens = [i.strip() for i in tokens if i.strip()]
    batch.append(tokens)
    
config = {
    'iterations': 2,
    'min_probability': 0.5,
    'min_error_probability': 0.7,
    'add_spell_check': False,
    'debug': True,
    'case_sensitive': True,
}
# batch call
preds, probs, idxs_batch, inter_pred, error_probs, cnt = model.handle_batch(batch, config=config, debug=True)

print(batch)
# print(inter_pred)
print(preds)
print(error_probs)
print()

for curr_idxs, curr_probs in zip(idxs_batch, probs):
    for iter_idxs, iter_probs in zip(curr_idxs, curr_probs):
        print(iter_idxs)
        print([model.vocab.get_token_from_index(i, namespace='labels') for i in iter_idxs])
        print(iter_probs)
        print()

[['England', 'coach', 'Andy', 'Robinson', 'is', 'already', 'without', 'centre', 'Will', 'Greenwood', 'and', 'flanker', 'Richard', 'Hill', 'while', 'fly', '-', 'half', 'Jonny', 'Wilkinson', 'is', 'certain', 'to', 'miss', 'at', 'least', 'the', 'first', 'two', 'games', '.']]
[['England', 'coach', 'Andy', 'Robinson', 'is', 'already', 'without', 'centre', 'Will', 'Greenwood', 'and', 'flanker', 'Richard', 'Hill', 'while', 'fly', '-', 'half', 'Jonny', 'Wilkinson', 'is', 'certain', 'to', 'miss', 'at', 'least', 'the', 'first', 'two', 'games', '.']]
[[0.616240382194519]]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
['$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$APPEND_,', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP', '$KEEP']
[0.9962007403373718, 0.9993681311607361, 

In [17]:
detail = model.generate_correct_detail(text, batch[0], preds[0], idxs_batch[0], probs[0], config)

In [18]:
detail

[['Hill', 'Hill while', [89, 93]]]

In [26]:
final_batch = ['England', 'coach', 'Andy', 'Robinson', 'is', 'already', 'without', 'centre', 'Will', 'Greenwood', 'and', 'flanker', 'Richard', 'Hill', 'while', 'fly', '-', 'half', 'Jonny', 'Wilkinson', 'is', 'certain', 'to', 'miss', 'at', 'least', 'the', 'first', 'two', 'games', '.']
len(final_batch)

31

In [27]:
model.update_final_batch(batch, [0], )

TypeError: update_final_batch() missing 2 required positional arguments: 'pred_batch' and 'prev_preds_dict'

In [37]:
preds[0]

['England',
 'coach',
 'Andy',
 'Robinson',
 'is',
 'already',
 'without',
 'centre',
 'Will',
 'Greenwood',
 'and',
 'flanker',
 'Richard',
 'Hill',
 'while',
 'fly',
 '-',
 'half',
 'Jonny',
 'Wilkinson',
 'is',
 'certain',
 'to',
 'miss',
 'at',
 'least',
 'the',
 'first',
 'two',
 'games',
 '.']

In [38]:
batch[0]

['England',
 'coach',
 'Andy',
 'Robinson',
 'is',
 'already',
 'without',
 'centre',
 'Will',
 'Greenwood',
 'and',
 'flanker',
 'Richard',
 'Hill',
 'while',
 'fly',
 '-',
 'half',
 'Jonny',
 'Wilkinson',
 'is',
 'certain',
 'to',
 'miss',
 'at',
 'least',
 'the',
 'first',
 'two',
 'games',
 '.']